# 命名實體識別 (NER)

此筆記本來自 [AI for Beginners Curriculum](http://aka.ms/ai-beginners)。

在這個範例中，我們將學習如何在 [命名實體識別的註解語料庫](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) 資料集上訓練 NER 模型。在開始之前，請下載 [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) 文件到當前目錄。


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## 準備資料集

我們將從將資料集讀入資料框開始。如果您想了解更多有關使用 Pandas 的資訊，請參考我們的 [初學者資料科學課程](http://aka.ms/datascience-beginners)中的[資料處理課程](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python)。


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


讓我們獲取唯一標籤並創建查找字典，以便我們可以用來將標籤轉換為類別編號：


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

現在我們需要對詞彙做同樣的處理。為了簡化，我們將在不考慮詞頻的情況下創建詞彙；在現實中，你可能會想使用 Keras 向量化工具，並限制詞彙的數量。


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

我們需要創建一個句子數據集用於訓練。讓我們遍歷原始數據集，並將所有單個句子分離到 `X`（單詞列表）和 `Y`（標記列表）：


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

為了簡化，我們將用 0 代幣填充所有句子到最大長度。在現實生活中，我們可能會想使用更聰明的策略，僅在一個小批次內填充序列。


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## 定義標記分類網絡

我們將使用兩層雙向 LSTM 網絡進行標記分類。為了對最後一層 LSTM 的每個輸出應用密集分類器，我們將使用 `TimeDistributed` 結構，該結構會在每一步中將相同的密集層複製到 LSTM 的每個輸出：


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

注意，在這裡我們為數據集明確指定了 `maxlen`——如果我們希望網絡能夠處理可變長度的序列，那麼在定義網絡時需要更加巧妙。

現在讓我們訓練模型。為了加快速度，我們只訓練一個周期，但您可以嘗試訓練更長的時間。此外，您可能希望將部分數據集分離作為訓練數據集，以觀察驗證準確性。


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## 測試結果

現在讓我們看看實體識別模型在一個範例句子上的表現：


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## 重點

即使是簡單的 LSTM 模型在命名實體識別（NER）上也能展現出不錯的效果。然而，若想獲得更好的結果，您可能需要使用大型的預訓練語言模型，例如 BERT。使用 Huggingface Transformers 庫訓練 BERT 用於 NER 的方法已在[此處](https://huggingface.co/course/chapter7/2?fw=pt)進行了說明。



---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原文文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤讀概不負責。
